In [2]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from netgraph import Graph
import statistics
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.decomposition import PCA
#from umap import UMAP  
from sklearn.manifold import MDS
import seaborn as sns
from scipy.stats import pearsonr
import utils

In [ ]:
D_data = pd.read_csv('data/Denmark/Raw/P19_22_DK.csv')
USA_data = pd.read_csv('data/USA/filtered_USA_votes.csv')


In [ ]:
D_PA, D_PAV, D_PAVP, D_PP = utils.dict_create(D_data, 'aktørid','party','afstemningid','typeid_x')
US_PA,US_PAV,US_PAVP,US_PP = utils.dict_create(USA_data,'icpsr','party_code','rollnumber','cast_code')


In [7]:
D_thres = np.average(utils.calc_thres(D_PP,D_PA,D_PAV))

In [8]:
edgelist = utils.edgelist_calc(D_PA,D_PAV, D_thres)
df = pd.DataFrame(edgelist, columns=["Source", "Target"])
df.to_csv("Denmark_edgelist.csv", index=False)
G_denmark = nx.from_edgelist(edgelist)

In [9]:
len(edgelist)

14938

In [10]:
list_of_dangle = [node for node in G_denmark.nodes if G_denmark.degree(node) == 0]
list_of_dangle

[]

In [11]:
U_thres = np.average(utils.calc_thres(US_PP, US_PA, US_PAV))

In [12]:
USA_inter= utils.calc_inter_edges(US_PP,US_PA,US_PAV)

In [13]:
U_edgelist = utils.edgelist_calc(US_PA, US_PAV, U_thres)
U_edgelist = (U_edgelist + USA_inter)
df = pd.DataFrame(U_edgelist, columns=["Source", "Target"])
df.to_csv("USA_edgelist.csv", index=False)
G_USA = nx.from_edgelist(U_edgelist)

In [14]:
list_of_dangle = [node for node in G_USA.nodes if G_USA.degree(node) == 0]
list_of_dangle

[]

Plotting the networks

In [3]:
import matplotlib.patches as mpatches

party_colors = {
    100: "blue",   # Democrat
    200: "red"     # Republican
}

congresses = ['095', '096', '097','098', '099', '100', '101', '102', '103','104',
              '105', '106', '107','108', '109', '110', '111', '112','113', '114',
              '115', '116','117','118']

for congress in congresses:
    # Load edge list
    edge_path = f"Data/USA/avg_party/H{congress}_USA_edgelist.csv"
    edges_df = pd.read_csv(edge_path)

    # Load node attributes (party_code)
    node_path = f"Data/USA/Filtered/H{congress}_filtered_USA_votes.csv"
    nodes_df = pd.read_csv(node_path)

    # Build graph — no edge_attr needed
    G = nx.from_pandas_edgelist(edges_df, source='Source', target='Target')

    # Map party codes to colors using 'icpsr'
    party_dict = nodes_df.set_index("icpsr")["party_code"].to_dict()
    color_map = [party_colors.get(party_dict.get(node, None), "gray") for node in G.nodes()]

    # Generate layout
    pos = nx.spring_layout(G, seed=42)

    # Draw the network
    plt.figure(figsize=(10, 10))
    nx.draw_networkx_nodes(G, pos, node_color=color_map, node_size=30)
    nx.draw_networkx_edges(G, pos, alpha=0.3, width=0.5)
    plt.title(f"Congress {congress} Network Graph")
    plt.axis("off")

    # Add legend
    legend_handles = [
        mpatches.Patch(color="blue", label="Democrat"),
        mpatches.Patch(color="red", label="Republican"),
        mpatches.Patch(color="gray", label="Other/Unknown")
    ]
    plt.legend(handles=legend_handles, loc="lower left", fontsize="small")

    # Save plot
    out_path = f"Images/USA_Networks_avg_party/congress_{congress}_network.png"
    plt.savefig(out_path, dpi=300, bbox_inches='tight')
    plt.close()
